### NLP team Data Challenge!!!
Hello interns! This is your first day working at BigBlueSteam Co!

Until 17.00 o’clock you have to use your Steam Game Review Dataset to create a model that predicts from text whether the user recommends a game title or not.

Until now our platform hosts reviews and game publishers accuse us of hosting comments that are often  unjustifiable and not based on evidence but on rage. Our company will use your model real-time so positive comments can be publicly available instantly. On the other hand, negative ones have to be detected in order to be reviewed before becoming public so we can avoid excessive language and unjustifiable accusations.

# Data Understanding

In [2]:
#Imports



In [ ]:
# EDA

# Data Preparation

In [ ]:
# EDA



## 1. Before anything else we need at some point all bad-language to be removed.

## 2.    It would be a great idea to accompany your script with a spell checking mechanism. (Demonstrate  it on single reviews not all dataset it may too take looong!!!)

## 3. Try to capture the comments with the most intense feelings of joy or anger.    Provide us with a model and script to run you classification from command line (you could keep a portion of data as a new csv to try it as external data)

## 4. Last but not least, any analysis on the reviews would be valuable and highly appreciated!
